In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
#convertir texto en un valor a una matrix, cada columna es una palabra
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import unicodedata
import pickle
import re
from sklearn.metrics import classification_report
from sklearn import metrics

In [104]:
df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 26.5+ KB


In [105]:
df.sample(10)

,url,is_spam
680,https://www.morningbrew.com/daily/2020/06/24,False
2724,https://apnews.com/ea0d2cff51aa296f47d35a20483...,False
1972,https://www.bookdepository.com/Love-Colour-Bol...,False
2118,https://docs.google.com/forms/d/13d2pf7UtxfKgl...,True
1301,https://ftw.usatoday.com/2020/06/nfl-jay-cutle...,False
656,https://www.morningbrew.com/daily/stories/2020...,False
2617,https://www.nytimes.com/2020/06/30/business/ad...,False
1468,https://www.thecut.com/2020/06/has-my-daughter...,False
1884,https://www.businesscasual.fm/,True
1031,https://www.youtube.com/watch?v=fXffdnOOE_Q,False


In [115]:
df['is_spam']=df['is_spam'].strip()

AttributeError: 'Series' object has no attribute 'strip'

In [113]:
print("spam count: " +str(len(df.loc[df.is_spam=='True'])))
print("not spam count: " +str(len(df.loc[df.is_spam=='True'])))
print(df.shape)
df['is_spam'] = df['is_spam'].astype(int)

spam count: 0
not spam count: 0
(2999, 2)


In [109]:
#Eliminamos los caracteres desconocidos
df['url']=df['url'].str.strip()
df['url']=df['url'].str.lower()
df

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,0
1,https://www.hvper.com/,0
2,https://briefingday.com/m/v4n3i4f3,0
3,https://briefingday.com/n/20200618/m#commentform,0
4,https://briefingday.com/fan,0
...,...,...
2994,https://www.smartcitiesworld.net/news/news/dee...,0
2995,https://www.youtube.com/watch,0
2996,https://techcrunch.com/2019/07/04/an-optimisti...,0
2997,https://www.technologyreview.com/2019/12/20/13...,0


In [110]:
df['url']=df['url'].str.replace('!','')
df['url']=df['url'].str.replace(',','')
df['url']=df['url'].str.normalize('NFKC') #ANALIZA TEXTO NO LATINO Y LO TRATA DE ARREGLAR
df['url']=df['url'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)
df.sample(10)

,url,is_spam
1262,https://w.npr.org/2020/06/26/883819835/obamaca...,0
1325,https://briefingday.com/fan,0
245,https://ftw.usatoday.com/2020/06/k-pop-stans-s...,0
2770,https://w.cnbc.com/2020/02/28/amid-testing-cap...,0
753,https://w.cbsnews.com/news/united-states-reces...,0
2188,https://w.businesscasual.fm/,0
2079,http://w.bbc.com/travel/story/20180621-the-unl...,0
1575,https://w.cnn.com/2020/06/21/us/guy-fieri-flav...,0
1443,https://w.youtube.com/watch?v=7fhi3djohl0,0
728,https://w.thrillist.com/travel/nation/travel-t...,0


In [111]:
def normalize_string(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else: 
        result=None
    return result

df['url']=df['url'].apply(normalize_string)
df.sample(10)

,url,is_spam
1541,https://briefingday.com/fan,0
2162,https://w.joinhoney.com/page/dr-us-simplified-...,0
1421,https://open.spotify.com/playlist/57rryt2rbbqa...,0
2190,https://w.morningbrew.com/emerging-tech/,0
2868,https://w1.nyc.gov/office-of-the-mayor/news/19...,0
2977,https://venturebeat.com/2020/06/30/ibm-uses-ai...,0
2111,https://thehustle.co/daily/,0
145,https://w.smithsonianmag.com/innovation/is-cov...,0
2346,https://w.bbc.com/news/health-53218704,0
2844,https://numlock.substack.com/p/numlock-sunday-...,0


In [112]:
X=df['review']
y=df['polarity']

KeyError: 'review'